In [ ]:
from pyincore import IncoreClient
from pyincore.analyses.socialvulnerability import SocialVulnerability

# Social vulnerability analysis for Douglas, Kansas census tracts (2019)

In [ ]:
# Parameters
year = 2019
state = "kansas"
county = "douglas"
census_geo_level = "tract"

In [ ]:
# Dataset required, can be computed from US Census data with pyincore_data 
national_vulnerability_feature_averages = "6242011d09b9bd46b433beb1"
social_vulnerability_demographic_factors = "6242017c5b9e61751de8e530"

In [ ]:
# Configure pyincore access and the analysis with the information above

client = IncoreClient()
social_vulnerability = SocialVulnerability(client)

social_vulnerability.set_parameter("result_name", "social_vulnerabilty")
social_vulnerability.set_parameter("year", year)
social_vulnerability.set_parameter("state", state)
social_vulnerability.set_parameter("county", county)
social_vulnerability.set_parameter("census_geo_level", census_geo_level)

social_vulnerability.load_remote_input_dataset("national_vulnerability_feature_averages",
                                               national_vulnerability_feature_averages)
social_vulnerability.load_remote_input_dataset("social_vulnerability_demographic_factors",
                                               social_vulnerability_demographic_factors)

In [ ]:
# Run social vulnerability damage analysis
result = social_vulnerability.run_analysis()

In [ ]:
# Retrieve result dataset
result = social_vulnerability.get_output_dataset("sv_result")

# Convert dataset to Pandas DataFrame
df = result.get_dataframe_from_csv()

# Display top 5 rows of output data
df.head()